In [ ]:
import pandas as pd 

## Loading the dataset :

df = pd.read_csv("zomato_outlet_final.csv")

In [3]:
## === DROPPING NOISE AND REPLACING NULL VALUES :

## Dropping "Links" columns as its useless :
df = df.drop(columns=["link"])

# replacing NULL values in deivery reviews with 0:
df["delivery_reviews"] = df["delivery_reviews"].fillna(0)

# keeping delivery_ratings as NULL (do nothing)

df.head(5)



,rest_name,rest_type,loc,dine_rating,dine_reviews,delivery_rating,delivery_reviews,cuisine,cost,liked
0,Mix@36 - The Westin\r\r\n ...,Lounge,"The Westin, Mundhwa",4.2,(707 Reviews),NaN,0,"North Indian,Continental,Asian,Finger Food,","Rs. 4,300","Cocktails, Chaat, Beer"
1,Kangan - The Westin\r\r\n ...,Fine Dining,"The Westin, Mundhwa",4.1,(344 Reviews),NaN,0,"North Indian,Mughlai,","Rs. 3,500","Biryani, Veg Platter, Dal Makhani, Butter Chic..."
2,Coriander Kitchen - Conrad Pune\r\r\n ...,Fine Dining,"Conrad Pune, Bund Garden Road",4.5,(541 Reviews),NaN,0,"Mediterranean,Asian,North Indian,","Rs. 3,200","Pasta, Panipuri, Waffles, Pizza, Salad, Chaat,..."
3,The Market - The Westin\r\r\n ...,Fine Dining,"The Westin, Mundhwa",4.0,(55 Reviews),NaN,0,"Asian,European,North Indian,","Rs. 3,200","Vanilla Ice Cream, Chaat, Sushi"
4,Vandaag - Vivanta Pune\r\r\n ...,"Pub,Casual Dining","Vivanta Pune, Hinjewadi",3.9,(113 Reviews),NaN,0,"Finger Food,Continental,","Rs. 3,100","Cocktails, Beer, Peri Peri Chicken, Fish, Veg ..."


In [4]:
## ==== RENAMING COLUMNS ====
df = df.rename(columns={
    "rest_name" : "restaurant_name",
    "loc" : "area",
    "cost" : "cost_for_two",
    "delivery_rating" : "delivery_rating",
    "delivery_reviews" : "delivery_reviews",
    "dine_rating" : "dine_rating",
    "dine_reviews" : "dine_reviews",
    "rest_type" : "restaurant_type",
    "cuisine" : "cuisine"
    
})

df.columns

Index(['restaurant_name', 'restaurant_type', 'area', 'dine_rating',
       'dine_reviews', 'delivery_rating', 'delivery_reviews', 'cuisine',
       'cost_for_two', 'liked'],
      dtype='str')

In [ ]:
## == CLEANING "COST FOR 2" COLUMN ==  imp => [^0-9] is being used for removing special char.

df['cost_for_two'] = (df['cost_for_two'].astype(str).str.replace(",","",regex=False).str.replace(r"[^0-9]","",regex=True))

df['cost_for_two'] = pd.to_numeric(df['cost_for_two'], errors="coerce")

## == CLEANING DINE-REVIEW COULMN :
df['dine_reviews'] = (df['dine_reviews'].astype(str).str.replace(",","",regex=False).str.replace(r"[^0-9]","",regex=True))

df['dine_reviews'] = pd.to_numeric(df['dine_reviews'], errors="coerce")

## === CLEANING EXTRA SPACES IN COLUMNS USING FOR LOOP :
text_cols = ['restaurant_name','area','restaurant_type','liked']
for col in text_cols :
    df[col] = (df[col].astype(str).str.replace(r'[\r\n]+',' ',regex=True).str.strip())
df.head(5)

,restaurant_id,restaurant_name,restaurant_type,area,dine_rating,dine_reviews,delivery_rating,delivery_reviews,cuisine,cost_for_two,liked
0,1,Mix@36 - The Westin,Lounge,"The Westin, Mundhwa",4.2,70700.0,NaN,0,"North Indian,Continental,Asian,Finger Food,",4300,"Cocktails, Chaat, Beer"
1,2,Kangan - The Westin,Fine Dining,"The Westin, Mundhwa",4.1,34400.0,NaN,0,"North Indian,Mughlai,",3500,"Biryani, Veg Platter, Dal Makhani, Butter Chic..."
2,3,Coriander Kitchen - Conrad Pune,Fine Dining,"Conrad Pune, Bund Garden Road",4.5,54100.0,NaN,0,"Mediterranean,Asian,North Indian,",3200,"Pasta, Panipuri, Waffles, Pizza, Salad, Chaat,..."
3,4,The Market - The Westin,Fine Dining,"The Westin, Mundhwa",4.0,5500.0,NaN,0,"Asian,European,North Indian,",3200,"Vanilla Ice Cream, Chaat, Sushi"
4,5,Vandaag - Vivanta Pune,"Pub,Casual Dining","Vivanta Pune, Hinjewadi",3.9,11300.0,NaN,0,"Finger Food,Continental,",3100,"Cocktails, Beer, Peri Peri Chicken, Fish, Veg ..."


In [ ]:
## GENERATING RESTARUNT_ID COLUMN :
df.insert(0, 'restaurant_id', range(1, len(df) + 1))


In [13]:
# === SAVING THE CLEANED CSV FILE AS restraunt.csv :
df.to_csv("restaurants.csv", index=False)


HERE IM GONNA CREATE A NEW TABLE i.e 'restaurant_cuisine' :

In [ ]:
cuisine_df = df[['restaurant_id', 'cuisine']].copy()

##  Replace NaN with empty string
cuisine_df['cuisine'] = cuisine_df['cuisine'].fillna('')

#  Split cuisines into lists
cuisine_df['cuisine'] = cuisine_df['cuisine'].str.split(',')

#  Convert lists into rows (MOST IMPORTANT STEP)
cuisine_df = cuisine_df.explode('cuisine')

#  Clean extra spaces
cuisine_df['cuisine'] = cuisine_df['cuisine'].str.strip()

#  Remove empty cuisine rows
cuisine_df = cuisine_df[cuisine_df['cuisine'] != '']

#  Save
cuisine_df.to_csv("restaurant_cuisine.csv", index=False)
